# Make sure that Miniconda3 and MFA are isntall and working prior to running these cells
- command "mfa align --help" should print help

In [ ]:
# copy wavs into aligned_pre
import os
import shutil

source_folder = r"../cmu_arctic/cmu_us_bdl_arctic/wav/"
destination_folder = r"./aligned_pre/"

# fetch all files
for file_name in os.listdir(source_folder):
    # construct full file path
    source = source_folder + file_name
    destination = destination_folder + "bdl_" + file_name
    # copy only files
    if os.path.isfile(source):
        if ".wav" in file_name:
            shutil.copy(source, destination)
            print('copied', file_name)

In [14]:
# create transcript files from metadata.csv
#lines = open('./ljs/LJSpeech-1.1/metadata.csv', 'r').readlines()
lines = open('../cmu_arctic/cmu_us_bdl_arctic/etc/txt.done.data', 'r').readlines()
from tqdm.auto import tqdm
for line in tqdm(lines):
    temp = line.split('"')
    transcript = temp[1]
    #print(transcript)
    fn = temp[0].split(' ')[1]
    #print(fn)
    ident = fn
    open(f'./aligned_pre/bdl_{ident}.txt', 'w').write(transcript)

# this is an example transcript for LJ001-0001.wav
#!cat ./wav/LJ001-0001.txt

100%|██████████| 1131/1131 [00:00<00:00, 3681.76it/s]


In [15]:
# download a pretrained english acoustic model, and english lexicon
!wget -q --show-progress -o ./english.zip= https://github.com/MontrealCorpusTools/mfa-models/raw/main/acoustic/english.zip
!wget -q --show-progress -o ./librispeech-lexicon.txt http://www.openslr.org/resources/11/librispeech-lexicon.txt

english.zip         100%[===================>]  14.05M  8.84MB/s    in 1.6s    
librispeech-lexicon 100%[===================>]   5.37M  1.68MB/s    in 3.2s    


In [16]:
# see: https://github.com/MontrealCorpusTools/Montreal-Forced-Aligner/pull/480
import re
lexicon = open("./librispeech-lexicon.txt").readlines()
sp = re.compile("\s+")
with open("./modified_librispeech-lexicon.txt", "w") as f:
    for line in lexicon:
        word, *phonemes = sp.split(line.strip())
        phonemes = " ".join(phonemes)
        f.write(f"{word}\t{phonemes}\n")

- Finish with command "mfa align -t ./temp -j 4 ./aligned_pre ./modified_librispeech-lexicon.txt ./english.zip ./aligned_post"